## Likelihood Ratio for MeerKAT ##

This notebook has the running of the Likelihood Ratio Code on MeerKAT.  Once the output from the Ridgeline Code has been produced this code can be used to determine the list of possible hosts for the MeerKAT data set.

In [ ]:
# Imports
from astropy.io import fits
from astropy.table import Table
import pandas as pd
from os.path import exists
from sklearn.neighbors import KernelDensity
import RidgelineFiles as RLF
from ridge_toolkitMK import DefineCutoutHDU, GetAvailableSources, GetCutoutArray
from SourceSearchMK import *

In [ ]:
if exists(RLF.psl) == False:
    print('Ridgelines not drawn.  Full Ridgeline code now running. Please wait output will show below.')
    %run 'MK - Ridgelines.ipynb'
else:
    print('Ridgeline information present. Please continue')

In [ ]:
TotalFluxCut = str(RLF.TFC)
available_sources = GetAvailableSources(TotalFluxCut)
print(available_sources.shape)

In [ ]:
# Load in the optical/IR and LOFAR catalogues, form tables and df and save as text files 
# Check columns esp on the Opt/IR
OptTable = TableOfSources(str(RLF.OptCat))
LofarTable = TableFromLofar(str(RLF.LofCat))
Lofardf = LofarTable.to_pandas()
Optdf = OptTable.to_pandas()
Optdf.to_csv(RLF.OptCatdf, columns = [str(RLF.IDW), str(RLF.IDP), str(RLF.PossRA), str(RLF.PossDEC), str(RLF.OptMagA), str(RLF.OptMagP)], header = True, index = False)

In [ ]:
# Set up source_list to be used in all of the following fuctions/cells
probfile = RLF.psl
source_list = GetSourceList(available_sources, probfile)
#source_list.remove('ILTJ163506.94+602836.1')
#source_list.remove('ILTJ092143.70+641518.6')
print(len(source_list))

In [ ]:
# Creating cutouts from the pw table so that it is easier to find the magnitudes for the 30 closest
# Only needs to be done once
for source in source_list:
    for asource in available_sources:
        if source == asource[0]:
            source_name = asource[0]
            lofarra = asource[4].astype(float)
            lofardec = asource[5].astype(float)
            sizepix = asource[6].astype(float)
            
            size = sizepix * RLF.ddel # convert size in pixels to degrees
            subcat = Optdf[(np.abs(Optdf[str(RLF.PossRA)] - lofarra) * np.cos(lofardec * np.pi / 180.0) < size) & (np.abs(Optdf[str(RLF.PossDEC)] - lofardec) < size)].copy()

            # Insert the uniform optical position error if required             
            
            subcat['raErr'] = np.where(np.isnan(subcat[str(RLF.OptMagP)]), RLF.UniWErr, RLF.UniLErr)
            subcat['decErr'] = np.where(np.isnan(subcat[str(RLF.OptMagP)]), RLF.UniWErr, RLF.UniLErr)
            
            subcat.to_csv(RLF.MagCO %source_name, columns = [str(RLF.IDW), str(RLF.IDP), str(RLF.PossRA), str(RLF.PossDEC), 'raErr', 'decErr', str(RLF.OptMagA), str(RLF.OptMagP)], header = True, index = False)


In [ ]:
# Looping through all successful sources to create the cutoutcat .txt files.  The distance away to form
# the sub-catalogue is set in RLConstants and is currently set to 1 arcmin RA and 0.5 arcmin DEC.
# Only needs to be done once

source_count = 0 ## Keeps track of where the loop is
for source in source_list:
    for asource in available_sources:
        if source == asource[0]:
            size = asource[6].astype(float)
            lofar_ra, lofar_dec = SourceInfo(source, LofarTable)[:2]
            #lofar_ra = asource[4].astype(float)
            #lofar_dec = asource[5].astype(float)
            subcat1 = CreateSubCat(OptTable, lofar_ra, lofar_dec)
    
            # Insert the uniform optical position error if required
            subcatdf = subcat1.to_pandas()

            # Insert the uniform optical position error if required             

            subcatdf['raErr'] = np.where(np.isnan(subcatdf[str(RLF.OptMagP)]), RLF.UniWErr, RLF.UniLErr)
            subcatdf['decErr'] = np.where(np.isnan(subcatdf[str(RLF.OptMagP)]), RLF.UniWErr, RLF.UniLErr)
            subcat2 = Table.from_pandas(subcatdf)
    
            cutoutcat = CreateCutOutCat(source, LofarTable, subcat2, lofar_ra, lofar_dec, size)
            source_count += 1
            print('Source Number = ' + str(source_count))

In [ ]:
# Create a table of R distance information from LOFAR Catalogue position
# Only needs to be done once
for source in source_list:
    CreateLDistTable(source, available_sources)

In [ ]:
# Find the 30 closest sources for each ridgeline
# Only needs to be done once
n = 30
NClosestDistances(source_list, available_sources, LofarTable, n)

In [ ]:
# Generating the likelihood ratios for all possible close sources, for each drawn
# ridgeline, using the R distance from LOFAR Catalogue position and the ridgeline.
# Only needs to be done once
LikelihoodRatios(source_list, available_sources)

In [ ]:
# Load in the three text files for each source, join all the table information together and save
# Only needs to be done once

for source in source_list:

    LofarLR = pd.read_csv(RLF.NLLR %source, header = 0)
    RidgeLR = pd.read_csv(RLF.NRLR %source, header = 0, usecols = ['Ridge_LR'])
    MagCutOut = pd.read_csv(RLF.MagCO %source, header = 0, usecols = [str(RLF.IDW), str(RLF.IDP), str(RLF.PossRA), str(RLF.OptMagA), str(RLF.OptMagP)])
    MagCutOut[str(RLF.PossRA)] = MagCutOut[str(RLF.PossRA)].apply(lambda x: round(x, 7))
            
    All_LR = LofarLR.join(RidgeLR['Ridge_LR'])
    All_LR['Multi_LR'] = All_LR['Lofar_LR'].astype(np.float64).multiply(All_LR['Ridge_LR'].astype(np.float64), axis = 'index')
        
    All_LR.columns=['LofarRDis', 'Lofar_LR', str(RLF.PossRA), str(RLF.PossDEC), 'Ridge_LR', 'Multi_LR']
    All_LR[str(RLF.PossRA)] = All_LR[str(RLF.PossRA)].apply(lambda x: round(x, 7))
            
    MagLR = All_LR.merge(MagCutOut, on = str(RLF.PossRA))
            
    MagLR.to_csv(RLF.NLRI %source, columns = ['LofarRDis', 'Lofar_LR', str(RLF.PossRA), str(RLF.PossDEC), 'Ridge_LR', 'Multi_LR', str(RLF.IDW), str(RLF.IDP), str(RLF.OptMagP), str(RLF.OptMagA)], header = True, index = False)


In [ ]:
# Functions I need to calculate the magnitude LR
# Make sure that the RA and DEC in RidgelineFiles has been changed to match the sky area being used
area = (np.deg2rad(RLF.LRAu) - np.deg2rad(RLF.LRAd)) * (np.sin(np.deg2rad(RLF.LDECu)) - np.sin(np.deg2rad(RLF.LDECd))) * np.rad2deg(3600)**2 

def get_nm(m, binscen, nm):
    find_nm = np.interp(m, binscen, nm)
    return find_nm

def get_qm(m, binscen, qm):
    find_qm = np.interp(m, binscen, qm)
    return find_qm

def get_LR(m, fr, binsn, binsq, nm, qm):
    n = get_nm(m, binsn, nm)
    q = get_qm(m, binsq, qm)
    LR = (q * fr) / n
    return LR

In [ ]:
# Set up the KDE for n(m) and q(m)

rbins = np.linspace(12, 30, 361)
rbinscen = (rbins[:-1] + rbins[1:]) / 2
W1bins = np.linspace(10, 23, 261)
W1binscen = (W1bins[:-1] + W1bins[1:]) / 2

# Form the separate band catalogues from DR2 to form the KDE for n(m)
Optdfr = Optdf[~np.isnan(Optdf[str(RLF.OptMagP)])]
Optdfw = Optdf[~np.isnan(Optdf[str(RLF.OptMagA)])]
Optdfr.reset_index(drop = True, inplace = True)  # Might not be needed but I have done it anyway
Optdfw.reset_index(drop = True, inplace = True)  # Might not be needed but I have done it anyway
kde_nmw1 = KernelDensity(kernel = 'gaussian', bandwidth = RLF.bw)
kde_nmw1.fit(np.array(Optdfw)[str(RLF.OptMagA)][:, None])
kde_nmr = KernelDensity(kernel = 'gaussian', bandwidth = RLF.bw)
kde_nmr.fit(np.array(Optdfr)[str(RLF.OptMagP)][:, None])

# Load in my host data to form the KDE for q(m)

hdata = pd.read_csv(str(RLF.DR1Hosts), header = 0, usecols = ['W1mag', 'i'])
hdata['r'] = hdata['i'].apply(lambda y: y + 0.33)
kde_hw1 = KernelDensity(kernel = 'gaussian', bandwidth = RLF.bw)
kde_hw1.fit(np.array(hdata)['W1mag'][:, None])
kde_hr = KernelDensity(kernel = 'gaussian', bandwidth = RLF.bw)
kde_hr.fit(np.array(hdata)['r'][:, None])

In [ ]:
# Then calculate the LR using the distance from the text files and save to text files

prob_qmhw1 = np.exp(kde_hw1.score_samples(np.array(W1binscen)[:, None]))
q_mhw1 = prob_qmhw1 * len(hdata['W1mag'])/np.sum(prob_qmhw1)
prob_nmw1 = np.exp(kde_nmw1.score_samples(W1binscen[:, None]))
n_mw1 = prob_nmw1/area * len(Optdfw[str(RLF.OptMagA)])/np.sum(prob_nmw1)

prob_qmhr = np.exp(kde_hr.score_samples(np.array(rbinscen)[:, None]))
q_mhr = prob_qmhr * len(hdata['r'])/np.sum(prob_qmhr)
prob_nmr = np.exp(kde_nmr.score_samples(rbinscen[:, None]))
n_mr = prob_nmr/area * len(Optdfr[str(RLF.OptMagP)])/np.sum(prob_nmr)

for source in source_list:
            
    MLR = pd.read_csv(RLF.NLRI %source, header = 0, usecols = ['LofarRDis', str(RLF.PossRA), str(RLF.PossDEC), str(RLF.IDW), str(RLF.IDP), str(RLF.OptMagP), str(RLF.OptMagA), 'Multi_LR'])
    MLR[str(RLF.LRMA)] = MLR.apply(lambda row: get_LR(np.float128(row[str(RLF.OptMagA)]), np.float128(row['Multi_LR']), W1binscen, W1binscen, n_mw1, q_mhw1), axis = 1).astype(np.float128)
    MLR[str(RLF.LRMP)] = MLR.apply(lambda row: get_LR(np.float128(row[str(RLF.OptMagP)]), np.float128(row['Multi_LR']), rbinscen, rbinscen, n_mr, q_mhr), axis = 1).astype(np.float128)

    MLR[str(RLF.LRMC)] = MLR[str(RLF.LRMA)].astype(np.float64).multiply(MLR[str(RLF.LRMP)].astype(np.float64), axis = 'index')
    
    MLR.to_csv(RLF.MLR %source, columns = ['LofarRDis', str(RLF.PossRA), str(RLF.PossDEC), str(RLF.IDW), str(RLF.IDP), str(RLF.OptMagP), str(RLF.OptMagA), str(RLF.LRMA), str(RLF.LRMP), str(RLF.LRMC)], header = True, index = False)

In [ ]:
# For each source in the list find the maximum combined LR and store all the information

def FindMax(source):
    info = pd.read_csv(RLF.MLR %source, header = 0, usecols = [str(RLF.PossRA), str(RLF.PossDEC), str(RLF.IDW), str(RLF.IDP), str(RLF.LRMC)])
    info[str(RLF.IDW)] = info[str(RLF.IDW)].map(lambda x: x.strip('b').strip("''"))
    info[str(RLF.IDP)] = info[str(RLF.IDP)].map(lambda x: x.strip('b').strip("''"))
    #info[str(RLF.ID3)] = info[str(RLF.ID3)].map(lambda x: x.strip('b').strip("''"))
    CP = info.loc[info[str(RLF.LRMC)].idxmax()].copy()
    CP['PossFail'] = np.where(CP[str(RLF.LRMC)] == 0, 1, 0)
    CP[str(RLF.LSN)] = source
    
    return CP

PossHosts = pd.concat([FindMax(source) for source in source_list], ignore_index = True, axis = 1)
PossHosts.columns = PossHosts.loc[str(RLF.LSN)]
PossHosts = PossHosts.drop(index = [str(RLF.LRMC), str(RLF.LSN)])
PossHostsT = PossHosts.transpose()
PossHostsT.to_csv(RLF.PossHosts, header = True, index = True)#,  columns = [str(RLF.LSN), str(RLF.PossRA), str(RLF.PossDEC), str(RLF.IDW), str(RLF.IDP), str(RLF.ID3), str(RLF.OptMagP), str(RLF.OptMagA)]

In [ ]:
# Number of sources with a 0 max LR and therefore would possibly be a failed LR
# or defined by being closest to LOFAR
print(np.sum(PossHostsT['PossFail']))